# Training Model: SVM

## 1. Importing libraries ...

In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.ml_prep import prep_ml, prep_ml_first20, save_ml_df, prep_ml_raw, average_trials_new, average_trials_and_participants_new
from jwlab.ml_prep_multigroup import prep_ml_multigroup
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from jwlab.constants import word_list

## 2. Importing Participants..

In [2]:
#participants = ["904", "905"]
#participants = ["904", "905", "906",  "908", "909", "910", "912", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]
participants = ["904", "905", "906", "908", "909","910", "912", "913", "914",  "916", "917", "921", "923", "927", "929", "930", "932"] 

#9m with >40 trials
#participants = [ "909", "912", "908", "913", "914", "916", "917", "919", "920", "921", "924","927", "930"]

#12m all
#participants = [ "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]

#12m with >40 trials
#participants = ["109", "111", "112", "115", "124"]

#all participants
# participants = ["904", "905", "906", "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932",
#               "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]



## 3. Set up averaging

In [3]:
X, y, good_trial_count = prep_ml_raw(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")


loaded


In [4]:
# #Randomize and split
# X[0][0] = X[0][0].sample(frac=1).reset_index(drop=True)
# len(X[0][0])
# fivefold_testsize = int(.20*len(X[0][0]))
# df_test = X[0][0][:fivefold_testsize]
# df_train = X[0][0][fivefold_testsize:]



num_indices = len(X[0][0])
fivefold_testsize = int(.20*num_indices)
test_indices = np.random.choice(num_indices-1, fivefold_testsize, replace=False)
df_test = []
df_train = []


    
X[0][0] = X[0][0].reset_index()

# #create new df with these indices and removing from orig
df_test.append(X[0][0].iloc[test_indices])
df_train.append(X[0][0].drop(X[0][0].index[test_indices]))
assert(len(df_train[0]) + len(df_test[0]) == len(X[0][0]))
df_test[0] = df_test[0].drop(columns=['index'], axis=1) 
df_train[0] = df_train[0].drop(columns=['index'], axis=1)





In [13]:
test_indices

array([628, 674, 804, 104, 484, 501, 326, 513, 727, 384, 310, 821, 793,
        48, 367, 446, 427, 287, 669, 722, 725, 562,  15, 400, 434, 599,
       578, 163, 132,  44, 453, 273, 791, 654, 425, 511, 374, 334, 190,
       391, 342, 613, 647,  77, 457, 490, 233, 292, 126,  51, 414, 569,
        97, 191, 272, 546, 757, 152,  45, 313, 394, 186, 679, 711, 790,
       832, 418, 767, 289, 728, 540, 588, 537,  43, 385, 261, 398, 475,
        82, 689,  92, 101, 420, 789, 251,   9, 778, 638, 753, 717, 801,
        35, 433, 451, 177, 312,  26, 405, 825, 285, 359, 222, 764, 196,
       666, 459, 720, 372, 556, 281, 576, 663, 416, 566,  66, 193, 387,
       582, 297, 835, 519, 760, 201, 355, 112, 834, 230, 691, 708, 756,
       527, 681, 826,  99,  29, 583, 577, 815, 247, 234, 293, 454, 447,
       176, 510,  23,  78,  90, 664, 237, 611,  28, 381, 158,  86, 820,
       271, 520, 476, 653, 633, 683, 295, 169, 283, 406, 411])

In [5]:
print(df_train.shape)
print(df_test.shape)

(834, 60002)
(208, 60002)


In [5]:
#calc averages for ps
num_participants = len(participants)
num_words = len(word_list)

#X_testp, y_testp, ps_p, words_p = average_trials_new(df_test, num_participants, num_words)
        
X_testp, y_testp, ps_p, words_p = average_trials_new(df_test[0], num_participants, num_words)
        
# change y to binary
y_testp[y_testp < 8] = 0
y_testp[y_testp >= 8] = 1

In [6]:
#calc averages for ps and trial
num_participants = len(participants)
num_words = len(word_list)

#X_testpt, y_testpt, ps_pt, words_pt = average_trials_and_participants_new(df_test, num_participants, num_words)


X_testpt, y_testpt, ps_pt, words_pt = average_trials_and_participants_new(df_test[0], num_participants, num_words)

# change y to binary
y_testpt[y_testpt < 8] = 0
y_testpt[y_testpt >= 8] = 1

In [7]:
# y_train = df_train['label']

# X_train = df_train.drop(columns = ['label', 'participant'])

y_train = df_train[0]['label']

X_train = df_train[0].drop(columns = ['label', 'participant'])




# change y to binary
y_train[y_train < 8] = 0
y_train[y_train >= 8] = 1

/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0.0: 344, 1.0: 327}

In [ ]:
df_train

In [ ]:
print(X_testpt.shape)

In [ ]:
#save_ml_df(X[0][0], "/Users/JennMacBook/Desktop/Studies/Animates EEG/7_Data/runningOffline/june17data12m.pkl")
np.savetxt('y12m.out', y[0][0])

## 3. b) Different groups

In [ ]:
X, y, good_trial_count = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
Xt, yt, good_trial_count_t = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
Xa, ya, good_trial_count_a = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")


In [ ]:
#good_trial_count

In [ ]:
(n, d) = X[0][0].shape
assert n == y[0][0].shape[0]


In [ ]:
d

In [ ]:
unique, counts = np.unique(y[0][0], return_counts=True)
dict(zip(unique, counts))

In [ ]:
(n, d) = Xt[0][0].shape
assert n == yt[0][0].shape[0]


n

In [ ]:
d

In [ ]:
unique, counts = np.unique(yt[0][0], return_counts=True)
dict(zip(unique, counts))

In [ ]:
(n, d) = Xa[0][0].shape
assert n == ya[0][0].shape[0]
n

In [ ]:
d

In [ ]:
unique, counts = np.unique(ya[0][0], return_counts=True)
dict(zip(unique, counts))

## 3. c) First 20 trials


In [ ]:
X, y, good_trial_count = prep_ml_first20(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
Xt, yt, good_trial_count_t = prep_ml_first20(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
Xa, ya, good_trial_count_a = prep_ml_first20(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")



In [ ]:
(n, d) = X[0][0].shape
assert n == y[0][0].shape[0]
n

In [ ]:
d

## 4. Setting up the SVM model ...

In [9]:
model = LinearSVC(C=1e-9, max_iter=5000)
#model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)

## 5. Training and testing the model ...

### 5.1. Train on raw, test on raw (validation)

In [10]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X_train, y_train)
np.mean(model.predict(X_train) != y_train)



0.30700447093889716

### 5.2. Train on raw, test on avg by trial (word repetition) 

In [11]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X_train, y_train)
np.mean(model.predict(X_testp) != y_testp)

0.4889705882352941

### 5.3. Train on raw, test on avg by word and ps 

In [12]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X_train, y_train)
np.mean(model.predict(X_testpt) != y_testpt)

0.375

In [16]:
from sklearn.model_selection import train_test_split

y_pred = model.predict(X_testpt)

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_testpt,y_pred)))
print('Precision Score : ' + str(precision_score(y_testpt,y_pred)))
print('Recall Score : ' + str(recall_score(y_testpt,y_pred)))
print('F1 Score : ' + str(f1_score(y_testpt,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_testpt,y_pred)))

Accuracy Score : 0.6875
Precision Score : 0.6666666666666666
Recall Score : 0.75
F1 Score : 0.7058823529411765
Confusion Matrix : 
[[5 3]
 [2 6]]


# 6 Subset analysis 

## 6.1 Generating random subsets of the chosen participant list

In [ ]:
participants_train, participants_test = train_test_split(participants,test_size=0.2)
print(len(participants_train), len(participants_test))

### 6.2.1 Create train and test sets: animates/inanimates

In [ ]:
#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

### 6.2.2 Create train and test sets - multi group

In [ ]:
#X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
#X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

#X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
#X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

### 6.2.3 Create train and test sets - first 20 trials

In [ ]:
#X_train, y_train, good_trial_count_train = prep_ml_first20(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
#X_test, y_test, good_trial_count_test = prep_ml_first20(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

#X_train, y_train, good_trial_count_train = prep_ml_first20(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
#X_test, y_test, good_trial_count_test = prep_ml_first20(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

X_train, y_train, good_trial_count_train = prep_ml_first20(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
X_test, y_test, good_trial_count_test = prep_ml_first20(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

In [ ]:
(n, d) = X_train[0][0].shape
assert n == y_test[0][0].shape[0]
n

In [ ]:
X_train[0][0].shape

### 6.2.4 Classification

In [ ]:
#model = SVC(gamma=.001, kernel = 'rbf', C=1e-6)
model = SVC(kernel = 'rbf')
model.fit(X_train[0][0], y_train[0][0])
np.mean(model.predict(X_test[0][0]) != y_test[0][0])

## 6.3 Monte Carlo Classification

In [ ]:

errorScores = []

# r iterations of a 5 fold
for r in range(2):
    kgp = np.array_split(participants, 5)

    for i in range(5):
        participants_test = kgp[i]
        participants_train = np.concatenate((kgp[(i+1)%5], kgp[(i+2)%5],kgp[(i+3)%5],kgp[(i+4)%5] ), axis=0)
        
        #X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
        #X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

        X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
        X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")


        #X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
        #X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

    
        #model = SVC(gamma=.001, kernel = 'rbf', C=1e-06)
        model = LinearSVC(C=1e-9, max_iter=5000)
        model.fit(X_train[0][0], y_train[0][0])
        errorScore = np.mean(model.predict(X_test[0][0]) != y_test[0][0])
        errorScores.append(errorScore)
    
errorScores



errorScores

In [ ]:
np.mean(errorScores)

In [ ]:
np.std(errorScores)

## 6.4 Alternate accuracy measurements

In [ ]:
from sklearn.model_selection import train_test_split

y_pred = model.predict(X_test[0][0])

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test[0][0],y_pred)))
print('Precision Score : ' + str(precision_score(y_test[0][0],y_pred)))
print('Recall Score : ' + str(recall_score(y_test[0][0],y_pred)))
print('F1 Score : ' + str(f1_score(y_test[0][0],y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test[0][0],y_pred)))

### 7 Optimization

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]}
svc = SVC()
model = GridSearchCV(svc, parameters, verbose=True)
model.fit(X_train[0][0], y_train[0][0])

model.cv_results_

In [ ]:
model.best_score_

In [ ]:
model.best_estimator_

# Cross Validation (5 fold)

In [ ]:
Xfirst = X[0][0]
yfirst = y[0][0]
# Xfirst = Xt[0][0]
# yfirst = yt[0][0]
# Xfirst = Xa[0][0]
# yfirst = ya[0][0]

Xfirst['label'] = yfirst
Xfirst = Xfirst.sample(frac=1).reset_index(drop=True)
ys = Xfirst['label']
Xs = Xfirst.drop(columns=['label'])


In [ ]:

model = LinearSVC(max_iter=5000, C=1e-9)
#model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)

In [ ]:
scores = cross_val_score(model, X[0][0], y[0][0], cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
scores = cross_val_score(model, Xs, ys, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Repeated N-Folds

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

testScores = []

X = X[0][0]
y = y[0][0]

# X = Xt[0][0]
# y = yt[0][0]

rkf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2652124)
for train_index, test_index in rkf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = LinearSVC(C=1e-9, max_iter=5000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    testScore = accuracy_score(y_test,y_pred)
    testScores.append(testScore)


testScores

In [ ]:
print(X.shape)

In [ ]:
print(X_train.shape)

In [ ]:
print(np.mean(testScores))
print(np.std(testScores))

In [ ]:
X.shape

In [ ]:
X_train.shape